# Set Up

Here is the standard bit of code that will set up the Intelligent Plant App Store and Data Core API clients for a notebook

In [1]:
import json

import intelligent_plant.session_manager as session_manager
import intelligent_plant.utility as utility

# Remeber to enable the device code flow in the app store app registration

#load the json config file with the app information
with open('config.json') as json_config_file:
    config = json.load(json_config_file)

    app_id = config['app']['id']
    app_secret = config['app']['secret']
    base_url = config['app_store']['base_url']


app_store = session_manager.load_session_or_login(app_id, app_secret, scopes=['DataRead'], base_url=base_url)

data_core = app_store.get_data_core_client()

# App Store

## Get Data Core Client

The app store client contains a function `get_data_core_client(..)` that instantiates a data core client with the same access information as that app store client. This is used to get the data core client needed to make data queries.

In [2]:
app_store.get_data_core_client()

## Get User Info

The `get_user_info()` function which can be used to get the information of the currently logged in user.

In [3]:
app_store.get_user_info()

{'ApplicationUserId': '48bafa11-ce56-4bdc-9e5a-088ee1c36f56',
 'UserId': '48bafa11-ce56-4bdc-9e5a-088ee1c36f56',
 'FirstName': 'Ross',
 'LastName': 'Kelso',
 'FormattedName': 'Ross RK. Kelso',
 'Industry': None,
 'Org': 'Intelligent Plant',
 'LastLogIn': '2024-04-29T10:18:15.117',
 'PicUrl': 'https://appstore.intelligentplant.com/profilepic/48bafa11-ce56-4bdc-9e5a-088ee1c36f56',
 'PicETag': None,
 'InfoAll': None,
 'VerificationStatus': 'Verified',
 'CreditBalance': 'NaN',
 'Email': None,
 'HasSharedApp': False,
 'HasSharedAppStore': False}

# Data Core

To get the data core client object use the `app_store.get_data_core_client()` function.

In [4]:
data_core = app_store.get_data_core_client()

**These examples use IP Datasource 2 as the data source please make sure that it is authorized for the Jupyter app**

https://appstore.intelligentplant.com/Security/Apps

## Get Data Sources

The get data sources method can be used to query the data sources available to the currently logged in user.

Returns:
A list of of available data sources and their meta data.

**When making other queries you want to use the fully qualified data source name**

```python
datasource["Name"]["QualifiedName"]
```

In [5]:
data_core.get_data_sources()

[{'Host': None,
  'SupportedFeatures': 'TagSearch, ReadTagValues, WriteTagValues, ValuePushNotifications, AssetModelBrowsing, AnnotationManagement',
  'Name': {'Name': 'Alarm Analysis Demo',
   'Namespace': 'FCBB05262EADC0B147746EE6DFB2B3EA5C272C33C2C5E3FE8F473D85529461CA',
   'QualifiedName': 'FCBB05262EADC0B147746EE6DFB2B3EA5C272C33C2C5E3FE8F473D85529461CA.Alarm Analysis Demo',
   'DisplayName': 'Alarm Analysis Demo (Demo Data Sources - IPABZAPP0009)',
   'Aliases': []},
  'TypeName': 'DataCore.Gestalt.GestaltBridgeDriver',
  'Description': 'Includes Alarm and Event analysis from varous simulated plant simulations.',
  'Status': {'RunningStatus': 'Running',
   'HealthStatus': {'IsHealthy': True, 'Properties': []},
   'IsInitialised': True,
   'IsDisabled': False,
   'DebugMode': False,
   'UtcStartupTime': '2025-02-10T04:00:32.3207132Z',
   'UtcLastModified': '2025-02-10T14:31:25.4250918Z',
   'Uptime': '10:30:53.1043786',
   'Messages': [],
   'Properties': [{'Name': 'Data Core Vers

In [6]:
dsn = 'FCBB05262EADC0B147746EE6DFB2B3EA5C272C33C2C5E3FE8F473D85529461CA.Oil Co Demo'

## Get Tags

The `get_tags(..)` function is used to query a data source for the available tags.

A tag is the name and meta data of a single data stream.

Tag searches are paginated to prevent the request sizes from getting too large.

Parameters:
- `dsn` The fully qualified name of the data source you want to query, see `get_data_sources(..)` for more information
- `page` (Optional, default = 1) The number of the results page you would like to get
- `page_size` (Optional, default = 20) The number of results to return per page
- `filters` (Optional, default = {}) A dictionary of filters to apply where the key is the field name (e.g. name, description, unit) and the value is the filter to apply (which can include the wildcards `?` and `*`)

Wildcards:
- `?` Any single character
- `*` Ay number (including 0) of any character

Returns:
A list of available tag objects

Data source information is the in 'DataSourceName' dictionary.

The tag information is in the 'Tags' list.

**In data queryies you want to use the values of the tag ID field**

In [7]:
data_core.get_tags(dsn)

[{'Id': 'AT-100143.IND_EU',
  'Name': 'AT-100143.IND_EU',
  'Description': '',
  'UnitOfMeasure': '',
  'Labels': None,
  'Properties': {'Is Numeric': {'Name': 'Is Numeric',
    'Category': 'General',
    'Description': 'Indicates if the tag contains any numeric values in the CSV data.',
    'DisplayIndex': 0,
    'Value': True,
    'IsReadOnly': True},
   'Raw Sample Count': {'Name': 'Raw Sample Count',
    'Category': 'General',
    'Description': 'The number of raw samples for the tag in the CSV data that was loaded.',
    'DisplayIndex': 0,
    'Value': 4320,
    'IsReadOnly': True},
   'UTC Earliest Raw Sample Time': {'Name': 'UTC Earliest Raw Sample Time',
    'Category': 'General',
    'Description': 'The earliest UTC time stamp in the CSV data that contains values for the tag.',
    'DisplayIndex': 0,
    'Value': '2022-01-01T00:00:00Z',
    'IsReadOnly': True},
   'UTC Latest Raw Sample Time': {'Name': 'UTC Latest Raw Sample Time',
    'Category': 'General',
    'Description':

## Getting Data

There are multiple different functions that are used to query datqa sources depending on the type of data you want to get and how it should be processed.

### Tag Map

All of the data query functions take a `tags` parameter. This parameter is used to specify which tags you want to query (and the data source they exist on).

The tag map should be a dictionary where the data source name is the key and the value is a list of tags on that data source.

For example to query the "hed and "flow" tags on IP Data source and the "Air Scour Blowers_Outlet_PI" and "Antifoam Inj Pump B_PI" tags on IP Data source 2 the `tags` parameter would be:

In [8]:
tags = {
    "IP Datasource": [
        "head",
        "flow"
    ],
    "IP Datasource 2": [
        "Air Scour Blowers_Outlet_PI",
        "Antifoam Inj Pump B_PI"
    ]
}

### Time Stamps

Data core allows you to use absolute or relative time stamps in your data queries.

When making a query the time parameters should be a string in one of these formats.

#### Absolute Time Stamps

Absoulte time stamps are in the ISO 8601 format:

In [9]:
from datetime import datetime, timezone
datetime.now(timezone.utc).isoformat()

'2025-02-10T14:31:16.603245+00:00'

#### Relative Time Stamps

Relative timestamps can be used to specify a time from the current time.

The `*` character indicates now and the `-` is used to subtract time from it.

Some examples:
- `"*"` is now 
- `"*-12h"` is 12 hours ago
- `"*-10d"` is 10 days ago
- `"*-1y"` is 1 year ago

### Responses

#### Tag Value

All of the get data function return values using the tag value objects that looks like this:

```python
{
    "TagName": "<tag name>",
    "UtcSampleTime": "<yyyy-MM-ddTHH:mm:ss.fffZ>",
    "NumericValue": "<some number>",
    "TextValue": "<some text>",
    "IsNumeric": true | false,
    "Status": "Good" | "Bad" | "Uncertain",
    "Unit": "<some unit>",
    "Notes": "<optional notes>",
    "Error": "<optional error message>",
    "HasError": true | false,
    "Properties": {
        "prop1": <some value>,
        ...
        "propN": <some value>
    }
}
```

#### Historical Tag Value

Historical data requests return data in a historical tag values object:

```python
{
    "TagName": "<tag name>",
    # Display hint when visualizing the data i.e straight line between points or trailing/leading edge
    "DisplayType": "Unspecified" | "Interpolate" | "TrailingEdge" | "LeadingEdge",
    "Values": [
        <DataCoreTagValue>,
        ...
        <DataCoreTagValue>
    ]
}
```

#### Response Dictionary

All responses organise the tag values in a dictionary where the key is the data source name and the value is a nested dictionary. The nested dictionary has the tag name as the key and the tag value object (for snapshot) or historical tag value object as the value.

```python
{
    "data source name 1": {
        "tag_name_1_1": <DataCoreTagValue or DataCoreHistoricalTagValue>,
        ...
        "tag_name_1_N": <DataCoreTagValue or DataCoreHistoricalTagValue>
    },
    "data source name N": {
        "tag_name_N_1": <DataCoreTagValue or DataCoreHistoricalTagValue>,
        ...
        "tag_name_N_N": <DataCoreTagValue or DataCoreHistoricalTagValue>
    }
}
```

### Get Plot Data

Plot data should be used when you want to draw a plot.

The way plot data is processed a special type of interploation that ensures that the maximum and minium values are preserved so that you don't lose an peaks.

To query plot data use the `get_plot_data(..)` function.

Parameters:
- `tags` The tag map of tags you want to quer (see the Tag Map section for more details)
- `start_time` The start time of the query range (see the Time Stamps section for more details)
- `end_time` The end time of the query range see the Time Stamps section for more details)
- `intervals` The number of intervals the time range should be divided into

Returns:

A response dictionary as decscribed in the Responses section, with 2 nested dictionarys of data source name and tag name with a historical tag value object as the value.

In [10]:
data_core.get_plot_data({dsn: ["Air Scour Blowers_Outlet_PI"]}, "*-10d", "*", 10)

{'FCBB05262EADC0B147746EE6DFB2B3EA5C272C33C2C5E3FE8F473D85529461CA.Oil Co Demo': {'Air Scour Blowers_Outlet_PI': {'TagName': 'Air Scour Blowers_Outlet_PI',
   'DisplayType': 'Interpolate',
   'Values': [{'TagName': 'Air Scour Blowers_Outlet_PI',
     'UtcSampleTime': '2025-01-31T14:31:25.6754254Z',
     'NumericValue': 0.0004354868897532797,
     'IsNumeric': True,
     'TextValue': None,
     'Status': 'Good',
     'Unit': '',
     'Notes': 'Interpolated using samples @ 2025-01-31T14:30:00.000Z and 2025-01-31T14:40:00.000Z.',
     'Error': None,
     'HasError': False,
     'Properties': {}},
    {'TagName': 'Air Scour Blowers_Outlet_PI',
     'UtcSampleTime': '2025-01-31T14:40:00Z',
     'NumericValue': 0.000355644,
     'IsNumeric': True,
     'TextValue': None,
     'Status': 'Good',
     'Unit': '',
     'Notes': None,
     'Error': None,
     'HasError': False,
     'Properties': {}},
    {'TagName': 'Air Scour Blowers_Outlet_PI',
     'UtcSampleTime': '2025-02-01T00:20:00Z',
   

### Get Processed Data

The get process data function is used to fecth data with a certain function applied. This allows the samples to have a consistent interval.

To query processed data use the `get_processed_data(..)` function.

Parameters:
- `tags` The tag map of tags you want to quer (see the Tag Map section for more details)
- `start_time` The start time of the query range (see the Time Stamps section for more details)
- `end_time` The end time of the query range see the Time Stamps section for more details)
- `sample_interval` A string representing the length of time each sample should represent, this uses a similar format to the realtive time stamps e.g. `"1d"`, `"1h"` etc.
- `data_function` The data processing function to use for the query. INTERP/AVG/MIN/MAX

Returns:

A response dictionary as decscribed in the Responses section, with 2 nested dictionarys of data source name and tag name with a historical tag value object as the value.

In [11]:
data_core.get_processed_data({dsn: ["Air Scour Blowers_Outlet_PI"]}, "*-10d", "*", "1d", "interp")

{'FCBB05262EADC0B147746EE6DFB2B3EA5C272C33C2C5E3FE8F473D85529461CA.Oil Co Demo': {'Air Scour Blowers_Outlet_PI': {'TagName': 'Air Scour Blowers_Outlet_PI',
   'DisplayType': 'Interpolate',
   'Values': [{'TagName': 'Air Scour Blowers_Outlet_PI',
     'UtcSampleTime': '2025-01-31T14:31:25.800376Z',
     'NumericValue': 0.0004354674926303867,
     'IsNumeric': True,
     'TextValue': None,
     'Status': 'Good',
     'Unit': '',
     'Notes': 'Interpolated using samples @ 2025-01-31T14:30:00.000Z and 2025-01-31T14:40:00.000Z.',
     'Error': None,
     'HasError': False,
     'Properties': {}},
    {'TagName': 'Air Scour Blowers_Outlet_PI',
     'UtcSampleTime': '2025-02-01T14:31:25.800376Z',
     'NumericValue': 0.0009678505313293734,
     'IsNumeric': True,
     'TextValue': None,
     'Status': 'Good',
     'Unit': '',
     'Notes': 'Interpolated using samples @ 2025-02-01T14:30:00.000Z and 2025-02-01T14:40:00.000Z.',
     'Error': None,
     'HasError': False,
     'Properties': {}},

### Get Raw Data

The get raw data function is used to get the raw values of the data in a certain time range, without any kind of data processing. Warning: this can causes extremely large amaounts of data t be returned.

To query raw data use the `get_raw_data(..)` function.

Parameters:
- `tags` The tag map of tags you want to quer (see the Tag Map section for more details)
- `start_time` The start time of the query range (see the Time Stamps section for more details)
- `end_time` The end time of the query range see the Time Stamps section for more details)
- `point_count` the maximum number of data points to return. 0 will fetch as many as possible. 

Returns:

A response dictionary as decscribed in the Responses section, with 2 nested dictionarys of data source name and tag name with a historical tag value object as the value.

In [12]:
data_core.get_raw_data({dsn: ["Air Scour Blowers_Outlet_PI"]}, "*-10d", "*", 10)

{'FCBB05262EADC0B147746EE6DFB2B3EA5C272C33C2C5E3FE8F473D85529461CA.Oil Co Demo': {'Air Scour Blowers_Outlet_PI': {'TagName': 'Air Scour Blowers_Outlet_PI',
   'DisplayType': 'TrailingEdge',
   'Values': [{'TagName': 'Air Scour Blowers_Outlet_PI',
     'UtcSampleTime': '2025-01-31T14:40:00Z',
     'NumericValue': 0.000355644,
     'IsNumeric': True,
     'TextValue': None,
     'Status': 'Good',
     'Unit': '',
     'Notes': None,
     'Error': None,
     'HasError': False,
     'Properties': {}},
    {'TagName': 'Air Scour Blowers_Outlet_PI',
     'UtcSampleTime': '2025-01-31T14:50:00Z',
     'NumericValue': 0.000262501,
     'IsNumeric': True,
     'TextValue': None,
     'Status': 'Good',
     'Unit': '',
     'Notes': None,
     'Error': None,
     'HasError': False,
     'Properties': {}},
    {'TagName': 'Air Scour Blowers_Outlet_PI',
     'UtcSampleTime': '2025-01-31T15:00:00Z',
     'NumericValue': 0.000169358,
     'IsNumeric': True,
     'TextValue': None,
     'Status': 'Goo

### Get Data At Times

The get data at times function allows you to qery data and specify the exact sample times you wuld like to have returned.

To query for specific times use the `get_data_at_times(..)` function.

Parameters:
- `tags` The tag map of tags you want to quer (see the Tag Map section for more details)
- `utc_sample_times` A list of UTC sample point to get the data for (in the ISO format)

Returns:

A response dictionary as decscribed in the Responses section, with 2 nested dictionarys of data source name and tag name with a historical tag value object as the value.

In [13]:
data_core.get_data_at_times({dsn: ["Air Scour Blowers_Outlet_PI"]}, [datetime.now(timezone.utc).isoformat()])

{'FCBB05262EADC0B147746EE6DFB2B3EA5C272C33C2C5E3FE8F473D85529461CA.Oil Co Demo': {'Air Scour Blowers_Outlet_PI': {'TagName': 'Air Scour Blowers_Outlet_PI',
   'DisplayType': 'TrailingEdge',
   'Values': [{'TagName': 'Air Scour Blowers_Outlet_PI',
     'UtcSampleTime': '2025-02-10T14:31:16.978055Z',
     'NumericValue': 0.000529686,
     'IsNumeric': True,
     'TextValue': None,
     'Status': 'Good',
     'Unit': '',
     'Notes': 'Repeated from raw value @ 2025-02-10T14:30:00.000Z',
     'Error': None,
     'HasError': False,
     'Properties': {}}]}}}

### Get Snapshot Data

The get the current value of some tags use the `get_snapshot_data(..)` function.

Parameters:
- `tags` The tag map of tags you want to quer (see the Tag Map section for more details)

Returns:

A response dictionary as decscribed in the Responses section, with 2 nested dictionarys of data source name and tag name with a tag value object as the value.

In [14]:
data_core.get_snapshot_data({dsn: ["Air Scour Blowers_Outlet_PI"]})

{'FCBB05262EADC0B147746EE6DFB2B3EA5C272C33C2C5E3FE8F473D85529461CA.Oil Co Demo': {'Air Scour Blowers_Outlet_PI': {'TagName': 'Air Scour Blowers_Outlet_PI',
   'UtcSampleTime': '2025-02-10T14:30:00Z',
   'NumericValue': 0.000529686,
   'IsNumeric': True,
   'TextValue': None,
   'Status': 'Good',
   'Unit': '',
   'Notes': None,
   'Error': None,
   'HasError': False,
   'Properties': {}}}}

## Data Frame Utility

The intelligent plant package includes a utility function that converts the response object into a pandas data frame.

**This function assumes that the timestamps of all the returned sample for each tag line up which isn't true for all queries and data functions**

Parmaters:
- `result` The result object from a data core request
- `include_dsn` (Optional, default=False) Whether data fram column names should include the data source name in the format `{dsn}.{tag name}`
- `force_numeric` (Optional, default=False) Whether all result data should be interpretted as numeric. By default each sample will be treated as string or numeric based on it's own `IsNumeric` property.
- `force_string` (Optional, default=False) Whether all result data should be interpretted as a string. By default each sample will be treated as string or numeric based on it's own `IsNumeric` property.

At most on of `force_numeric` and `force_string` can be true.

Returns a data frame of the same data with tag names as column names and the time stamp as the index.

Data frames can be used by a variety of data analysis and machine learning frameworks , see the other demo notebooks for more information

In [15]:
import intelligent_plant.utility as utility

In [16]:
resp = data_core.get_processed_data({dsn: ["Air Scour Blowers_Outlet_PI"]}, "*-30d", "*", "1d", "interp")
df = utility.query_result_to_data_frame(resp)
df

,Air Scour Blowers_Outlet_PI
2025-01-11 14:31:26.255273800+00:00,0.00053
2025-01-12 14:31:26.255273800+00:00,0.00013
2025-01-13 14:31:26.255273800+00:00,0.003876
2025-01-14 14:31:26.255273800+00:00,0.000253
2025-01-15 14:31:26.255273800+00:00,0.001885
2025-01-16 14:31:26.255273800+00:00,0.000385
2025-01-17 14:31:26.255273800+00:00,0.000296
2025-01-18 14:31:26.255273800+00:00,0.000432
2025-01-19 14:31:26.255273800+00:00,0.00047
2025-01-20 14:31:26.255273800+00:00,0.000661
